### Initialisation of variables:

In [1]:
import random
import numpy as np
import pandas as pd

edges = {(0,2),(2,0),(1,1),(1,2),(2,2),(2,3),(3,3),(3,4),(6,3),(4,6),(6,4),(6,6),(5,6),(5,5)} # Dictionary containing graph edges 
list_of_edges = list(edges)                                                                   # A list of the edges - for slicing
number_of_edges = len(edges)                                                                  # Number of edges
pageDict = {}                                                           
    
for edge in edges:
    pageDict[edge[0]] = 0                                                                     # adds the unique pages to dictionary

pages = list(pageDict)                                                                        # converts dictionary to a list
number_of_pages = len(pages)                                                                  # calculate number of pages in graph


### Define a getNeighbours function:
A page is passed as an argument into the method and a list of its adjacent neighbours is outputted.


In [2]:
# neigh = a list of neighbours
# edge = the edge currently being used from the list of edges

def getNeighbours(page):
    neigh=[]
    
    for x in range(number_of_edges):
        edge = list_of_edges[x]                 # Gets the specific edge from the list
        
        if edge[0] == page:                     # Checks the first value as this is where the edge comes from
            neigh.append(edge[1])               # This is where the edeg leads to. This page is appended to a list
    
    return neigh

### Testing the getNeighbours function:
When the function is applied to the page 1, the neighbouring pages are 1 and 2, and when the function is applied to page 6, the neighbouring pages are 3, 4, and 6. Therefore the function works.

In [3]:
print("Page 1 neighbours are: " + str(getNeighbours(1)))
print("Page 3 neighbours are: " + str(getNeighbours(6)))

Page 1 neighbours are: [2, 1]
Page 3 neighbours are: [4, 6, 3]


### The random walk method: 
A random page is selected as a starting node. If the current iteration is divisable by the tel variable (the frequency of teleportations) with no remainder, then the page is randomly teleported to any other page within the graph to prevent dead ends, otherwise the walker randomly selects one of the neighbouring pages to move to. The frequencies of the pages are stored in a dicitonary and then divided by the total number of iterations to give normalised values.

In [4]:
# iters = the number of iterations
# a = the teleportation probability
# edges = the edges within the graph
# currentPage = the page which the walker is currently at
# dict_counter = the frequencies for each of the pages
# pageDict = dictionary for unique pages

def randomwalk(edges, a, iters):
    
    pageDict = {}                                                           
    dict_counter = {}                                                       
    
    for edge in edges:
        pageDict[edge[0]] = 0                                               # adds the unique pages to dictionary

    pages = list(pageDict)                                                  # converts dictionary to a list
    number_of_pages = len(pages)                                            # calculate number of pages in graph
    currentPage = random.choice(pages)                                      # randomly selects any page in the graph to start    
    
    for page in range(number_of_pages):
        dict_counter[page] = 0                                              # initialise dictionary with 0's for page freqencies 

    for _ in range(iters):            
        if random.uniform(0,1) < a:                                         # random float number from 0 to 1 selected
            currentPage = random.choice(pages)                              # randomly selects any page in the graph
        else:
            currentPage = random.choice(getNeighbours(currentPage))         # randomly selects a neighboured page 
        dict_counter[currentPage] += 1                                      # adds 1 to the frequency of current page
            
    for page in range(len(dict_counter)):
        dict_counter[page] = dict_counter[page] / iters                     # normalises the frequencies

    return dict_counter

### Running the random walk method:
The method outputs the scores of each of the pages. It should be noted that the values will slightly change for this method each time it is run as it uses a randomiser. The dictionary is stored in the variable 'frequencies' so that it can be accessed later.

In [5]:
frequencies = randomwalk(edges, 0.14, 10000)
frequencies

{0: 0.0527, 1: 0.0354, 2: 0.1174, 3: 0.2534, 4: 0.2106, 5: 0.0303, 6: 0.3002}

### Pagerank function:

In [6]:
# matrix = list of every probability for the page connections - transition matrix
# T = an N x N matrix full of 1/N
# alpha = the teleportation probability
# bias = the additional value added to every probability
# dictionary for unique pages

def pagerank(edges, a, iters):
    
    pageDict = {}                                                           
    
    for edge in edges:
        pageDict[edge[0]] = 0                          # adds the unique pages to dictionary

    pages = list(pageDict)                             # converts dictionary to a list
    number_of_pages = len(pages)                       # calculate number of pages in graph
    matrix = []
    T = 1/number_of_pages
    alpha = a
    bias = T * alpha

           
    for page in range(number_of_pages):
        connectedPages = getNeighbours(page)           # gets neighbours of current page
        row = [0] * number_of_pages                    # initialising list with a 0 frequency for each page of length 'number_of_pages'  


        if len(connectedPages) == 1:
            prob = 1                                   # probability is 1 if there's only 1 connected page
        else:
            prob = 1/len(connectedPages)               # probabilities are 1/n where n = number of nighbours 

        prob = (((1-alpha)*prob) + bias)               # teleportation formula applied to every page that is not equal to 0  
    
        for x in connectedPages:
            row[x] = prob                              # the pages probabilities stored in the list of row

        for y in range(len(row)):                      
            if row[y] == 0:
                row[y] += bias                         # teleportation formula applied to every page that is equal to 0

        matrix.append(row)                             # appends the row to the matrix

    trans_matrix = np.array(matrix)                    # convert the list of rows to a numpy array so it can be reshaped
    trans_matrix.reshape(7,7)                          # reshaped the array to be 
    
    a = [T] * number_of_pages                          # initialises each of the pages in the 'a' matrix to 1/n (T)

    for i in range(iters):
        a = np.matmul(a, trans_matrix)                 # multiplies the two matrixes (a and trans_matrix) iters amount of times,
                                                       # updating the value of a each time
    return a

### Running the pagerank method:
The method outputs the scores of each of the pages. Unlike the random walk method, the values in the pagerank method will not change each time it is run. The final vector is stored in the variable 'vector' so that it can be accessed later.

In [7]:
vector = pagerank(edges, 0.14, 10000)
vector

array([0.05211042, 0.03508772, 0.11201311, 0.24561199, 0.21350156,
       0.03508772, 0.30658747])

### The following is used to output the ranking results in order of both methods:
Note that pages 1 and 5 will swap in the ranking with one another for the random walk method as they are both similar (they can loop back to themselves and have no incoming edges), and therefore the randomiser will have an impact upon there ranking order. The output helps to prove that both of the methods do converge to a steady state.

In [8]:
import operator

pagedict = {}                                                                              # dictionary for the pagerank results
randomdict = {}                                                                            # dictionary for the random walk results

for x in range(number_of_pages):
    pagedict[x] = vector[x]                                                                # copies the vector dictionary across 

sortingPage = sorted(pagedict.items(), key=operator.itemgetter(1), reverse=True)           # sorts the pagerank method results
sortingRandom = sorted(frequencies.items(), key=operator.itemgetter(1), reverse=True)      # sorts the random walk method results

for x in range(number_of_pages):
    currentPage = sortingPage[x]                                                           # the current pagerank score for output
    currentRandom = sortingRandom[x]                                                       # the current random walk score for output
    
    print("\033[1mRanking number " + str(x+1) + ": \n\033[0mRandom Walk Method:       \033[1mPage " + str(currentRandom[0]) + " : " + str(currentRandom[1]))
    print("\033[0mPagerank Method:          \033[1mPage " + str(currentPage[0]) + " : " + str(currentPage[1]) + "\n")
    

Ranking number 1: 
Random Walk Method:       Page 6 : 0.3002
Pagerank Method:          Page 6 : 0.3065874740538629

Ranking number 2: 
Random Walk Method:       Page 3 : 0.2534
Pagerank Method:          Page 3 : 0.24561198915656474

Ranking number 3: 
Random Walk Method:       Page 4 : 0.2106
Pagerank Method:          Page 4 : 0.21350156456609684

Ranking number 4: 
Random Walk Method:       Page 2 : 0.1174
Pagerank Method:          Page 2 : 0.1120131090365158

Ranking number 5: 
Random Walk Method:       Page 0 : 0.0527
Pagerank Method:          Page 0 : 0.05211042459046785

Ranking number 6: 
Random Walk Method:       Page 1 : 0.0354
Pagerank Method:          Page 5 : 0.0350877192982456

Ranking number 7: 
Random Walk Method:       Page 5 : 0.0303
Pagerank Method:          Page 1 : 0.03508771929824559



### A better way of visualising this can be seen below in a dataframe:

In [9]:
final_res = []                                    # A list of the pageranking scores for each of the iterations
col=[]                                            # List of names of the dataframe columns (the number of iterations) 
x = 12                                            # The number of iterations

for i in range(x):
    final_res.append(pagerank(edges, 0.14, i))    # Calls the pagerank function and appends the vector to the final_res list
    if i == 0:
        col.append("x")                           # The starting value of choosing any possible page 
    else:
        col.append("xP^" + str(i))                # The column name

trans_matrix = np.array(final_res)                # Puts array into a numpy array so it can be adapted with ease
trans_matrix = trans_matrix.reshape(x,7)          # Reshapes matrix so it becomes matrix rather than 1D array
trans_matrix = trans_matrix.transpose()           # Transposed to ensure the pages are listed as rows rather than columns
dataframe=pd.DataFrame(trans_matrix, columns=col) #Put into a dataframe for an easier visalisation 
dataframe

,x,xP^1,xP^2,xP^3,xP^4,xP^5,xP^6,xP^7,xP^8,xP^9,xP^10,xP^11
0,0.142857,0.060952,0.090302,0.070951,0.069383,0.062763,0.059877,0.057158,0.055556,0.054379,0.053626,0.053111
1,0.142857,0.081429,0.055014,0.043656,0.038772,0.036672,0.035769,0.035381,0.035214,0.035142,0.035111,0.035098
2,0.142857,0.245238,0.177735,0.172266,0.149173,0.139104,0.129622,0.124033,0.119926,0.117299,0.115502,0.114326
3,0.142857,0.163333,0.230837,0.236305,0.242033,0.244013,0.244933,0.245319,0.245486,0.245558,0.245589,0.245602
4,0.142857,0.122381,0.160535,0.185355,0.192650,0.201250,0.205056,0.208161,0.209930,0.211179,0.211963,0.212491
5,0.142857,0.081429,0.055014,0.043656,0.038772,0.036672,0.035769,0.035381,0.035214,0.035142,0.035111,0.035098
6,0.142857,0.245238,0.230563,0.247811,0.269216,0.279527,0.288975,0.294569,0.298675,0.301302,0.303098,0.304274


### Q3:
To determine that both methods result in a steady state can easily be done by simply setting the iteration values of the methods to a high number such as 10000. By doing so, this outputs the resulting ranking of 6, 4, 3, 2, 0, 1, 5 and this ranking does not change no matter how many times the methods are run (with the exception of pages 1 and 5 for the random walk method as previously explained). However, it's difficult to determine a specific number of iterations required for the random walk method to result in a steady state, as the randomness of the approach causes the page scores to change slightly each time the function is run. Therefore as the steady state of the pagerank is reached after approximately 7 iterations, this number of iterations should be used in the randomwalk method as a benchmark to test. The random walk method does not even converge to a steady state after 12 iterations and therefore the pagerank method is proven to converge faster.

### Q4:
Users cannot improve their page ranking score by changing the number of outgoing edges to other pages. If they were to do so, this would simply give their page a poorer ranking score as other pages which now have an additional incoming edge will be the pages that gain a better score from doing this instead. Pagerank uses the incoming edges to determine their rankings rather than the outgoing edges. Therefore if a page with a high page ranking score is pointing to their page, their page will then have a higher score as it is more likely to be accessed than a page with none or very few incoming edges from low frequently visited pages. Proof of this follows:    

This is the set of results which will be compared with the tests:

![Title](comparingResults.png)

As you can see with the first example below, the edges dictionary contains more outgoing links to other pages from page 6. It is therefore expected that the ranking score of page 6 should be a smaller value than the original 0.304274, whilst the other pages which now have an additional input edge (pages 1, 2 and 5) should have larger page ranking scores: 

In [10]:
import random
import numpy as np
import pandas as pd

edges = {(0,2),(2,0),(1,1),(1,2),(2,2),(2,3),(3,3),(3,4),(6,3),(4,6),(6,4),(6,6),(5,6),(5,5),    
         (6,1),(6,2),(6,5)}                       # The edges that are added to the original edges dictionary for testing 

pageDict = {}                                                           
    
for edge in edges:
    pageDict[edge[0]] = 0                         # adds the unique pages to dictionary

pages = list(pageDict)                            # converts dictionary to a list
number_of_pages = len(pages)                      # calculate number of pages in graph
list_of_edges = list(edges)                       # A list of the edges - for slicing
number_of_edges = len(edges)                      # Number of edges
dict_counter = {}                                 # Dictionary of pages & their scores
final_res = []                                    # A list of the pagerankings of each of the iterations
col=[]                                            # List of names of the dataframe columns (the number of iterations) 
x = 12                                            # The number of iterations

for i in range(x):
    final_res.append(pagerank(edges, 0.14, i))    # Calls the pagerank function
    if i == 0:
        col.append("x")                           # The starting value of choosing any possible page 
    else:
        col.append("xP^" + str(i))                # The column name

trans_matrix = np.array(final_res)                # Puts array into a numpy array so it can be adapted with ease
trans_matrix = trans_matrix.reshape(x,7)          # Reshapes matrix so it becomes matrix rather than 1D array
trans_matrix = trans_matrix.transpose()           # Transposed to ensure the pages are listed as rows rather than columns
dataframe=pd.DataFrame(trans_matrix, columns=col) #Put into a dataframe for an easier visalisation 
dataframe

,x,xP^1,xP^2,xP^3,xP^4,xP^5,xP^6,xP^7,xP^8,xP^9,xP^10,xP^11
0,0.142857,0.060952,0.096171,0.084393,0.087287,0.084245,0.084037,0.083245,0.083087,0.082921,0.082890,0.082869
1,0.142857,0.101905,0.096035,0.087621,0.084245,0.084072,0.084136,0.084555,0.084814,0.085009,0.085108,0.085161
2,0.142857,0.265714,0.224625,0.234722,0.224109,0.223383,0.220623,0.220072,0.219493,0.219385,0.219311,0.219316
3,0.142857,0.142857,0.189816,0.192340,0.196561,0.196613,0.196565,0.196145,0.195885,0.195691,0.195592,0.195538
4,0.142857,0.101905,0.113644,0.127947,0.129274,0.132368,0.132529,0.132899,0.132798,0.132769,0.132701,0.132669
5,0.142857,0.101905,0.096035,0.087621,0.084245,0.084072,0.084136,0.084555,0.084814,0.085009,0.085108,0.085161
6,0.142857,0.224762,0.183673,0.185356,0.194280,0.195248,0.197973,0.198530,0.199108,0.199215,0.199290,0.199284


The dataframe that is outputted proves this theory as the score is now 0.199284 whilst the scores of pages 1, 2 and 5 have improved. Another test to prove this can be seen below where page 6 has three more input edges than in the original graph. This therefore should have an improved page ranking score for page 6 (0.304274), whilst pages 1 (0.035098), 2 (0.114326) and 3 (0.245602) should have smaller page ranking scores.  

In [11]:
import random
import numpy as np
import pandas as pd

edges = {(0,2),(2,0),(1,1),(1,2),(2,2),(2,3),(3,3),(3,4),(6,3),(4,6),(6,4),(6,6),(5,6),(5,5),    
         (1,6),(2,6),(3,6)}                       # The edges that are added to the original edges dictionary for testing 

pageDict = {}                                                           
    
for edge in edges:
    pageDict[edge[0]] = 0                         # adds the unique pages to dictionary

pages = list(pageDict)                            # converts dictionary to a list
number_of_pages = len(pages)                      # calculate number of pages in graph
list_of_edges = list(edges)                       # A list of the edges - for slicing
number_of_edges = len(edges)                      # Number of edges
dict_counter = {}                                 # Dictionary of pages & their scores
final_res = []                                    # A list of the pagerankings of each of the iterations
col=[]                                            # List of names of the dataframe columns (the number of iterations) 
x = 12                                            # The number of iterations

for i in range(x):
    final_res.append(pagerank(edges, 0.14, i))    # Calls the pagerank function
    if i == 0:
        col.append("x")                           # The starting value of choosing any possible page 
    else:
        col.append("xP^" + str(i))                # The column name

trans_matrix = np.array(final_res)                # Puts array into a numpy array so it can be adapted with ease
trans_matrix = trans_matrix.reshape(x,7)          # Reshapes matrix so it becomes matrix rather than 1D array
trans_matrix = trans_matrix.transpose()           # Transposed to ensure the pages are listed as rows rather than columns
dataframe=pd.DataFrame(trans_matrix, columns=col) # Put into a dataframe for an easier visalisation 
dataframe

,x,xP^1,xP^2,xP^3,xP^4,xP^5,xP^6,xP^7,xP^8,xP^9,xP^10,xP^11
0,0.142857,0.050714,0.066123,0.047350,0.044716,0.040264,0.038701,0.037507,0.036952,0.036609,0.036432,0.036330
1,0.142857,0.060952,0.037473,0.030742,0.028813,0.028260,0.028101,0.028056,0.028043,0.028039,0.028038,0.028038
2,0.142857,0.214524,0.127210,0.114958,0.094250,0.086979,0.081428,0.078845,0.077250,0.076426,0.075953,0.075699
3,0.142857,0.132619,0.206726,0.206042,0.212302,0.213945,0.214755,0.215225,0.215400,0.215516,0.215563,0.215593
4,0.142857,0.101905,0.160603,0.178692,0.187586,0.193681,0.196055,0.197718,0.198448,0.198907,0.199132,0.199263
5,0.142857,0.081429,0.055014,0.043656,0.038772,0.036672,0.035769,0.035381,0.035214,0.035142,0.035111,0.035098
6,0.142857,0.357857,0.346851,0.378559,0.393562,0.400200,0.405191,0.407268,0.408694,0.409362,0.409772,0.409981


The outputted results prove this theory once again. The score of page 6 has increased to 0.409981, whilst the scores of 1, 2 and 3 have decreased to 0.028038, 0.075699, and 0.215593 respectively. 

# The code below used was used for testing purposes:

### The pagerank function which returns a transition matrix 

In [12]:
# matrix = list of every probability for the page connections - transition matrix
# T = an N x N matrix full of 1/N
# alpha = the teleportation probability
# bias = the additional value added to every probability
# dictionary for unique pages

def pagerank(edges, a, iters):
    
    pageDict = {}                                                           
    
    for edge in edges:
        pageDict[edge[0]] = 0                          # adds the unique pages to dictionary

    pages = list(pageDict)                             # converts dictionary to a list
    number_of_pages = len(pages)                       # calculate number of pages in graph
    matrix = []
    T = 1/number_of_pages
    alpha = a
    bias = T * alpha


    for page in range(number_of_pages):
        connectedPages = getNeighbours(page)           # gets neighbours of current page
        row = [0] * number_of_pages                    # initialising list with a 0 frequency for each page of length 'number_of_pages'  

        if len(connectedPages) == 1:
            prob = 1                                   # probability is 1 if there's only 1 connected page
        else:
            prob = 1/len(connectedPages)               # probabilities are 1/n where n = number of nighbours 


        for y in range(number_of_pages):
            for x in connectedPages: 
                if y == x:
                    row[x] = prob                      # adds each of the pages probabilities to the row list

        matrix.append(row)                             # appends the row to the matrix

    trans_matrix = np.array(matrix)                    # convert the list of rows to a numpy array so it can be reshaped
    trans_matrix.reshape(7,7)                          # reshaped the array to be 


    return trans_matrix

In [13]:
trans_matrix = pagerank(edges, 0.14, 100)

dataframe=pd.DataFrame(trans_matrix, columns=['0','1','2', '3', '4', '5', '6']) #Put into a dataframe for an easier visalisation 
dataframe

,0,1,2,3,4,5,6
0,0.00,0.000000,1.000000,0.000000,0.000000,0.0,0.000000
1,0.00,0.333333,0.333333,0.000000,0.000000,0.0,0.333333
2,0.25,0.000000,0.250000,0.250000,0.000000,0.0,0.250000
3,0.00,0.000000,0.000000,0.333333,0.333333,0.0,0.333333
4,0.00,0.000000,0.000000,0.000000,0.000000,0.0,1.000000
5,0.00,0.000000,0.000000,0.000000,0.000000,0.5,0.500000
6,0.00,0.000000,0.000000,0.333333,0.333333,0.0,0.333333


### The pagerank function which returns a transition matrix including teleporting 

In [14]:
# matrix = list of every probability for the page connections - transition matrix
# T = an N x N matrix full of 1/N
# alpha = the teleportation probability
# bias = the additional value added to every probability
# dictionary for unique pages

edges = {(0,2),(2,0),(1,1),(1,2),(2,2),(2,3),(3,3),(3,4),(6,3),(4,6),(6,4),(6,6),(5,6),(5,5)} # Dictionary containing graph edges 

def pagerank(edges, a, iters):
    
    pageDict = {}                                                           
    
    for edge in edges:
        pageDict[edge[0]] = 0                          # adds the unique pages to dictionary

    pages = list(pageDict)                             # converts dictionary to a list
    number_of_pages = len(pages)                       # calculate number of pages in graph
    matrix = []
    T = 1/number_of_pages
    alpha = a
    bias = T * alpha

           
    for page in range(number_of_pages):
        connectedPages = getNeighbours(page)           # gets neighbours of current page
        row = [0] * number_of_pages                    # initialising list with a 0 frequency for each page of length 'number_of_pages'  


        if len(connectedPages) == 1:
            prob = 1                                   # probability is 1 if there's only 1 connected page
        else:
            prob = 1/len(connectedPages)               # probabilities are 1/n where n = number of nighbours 

        prob = (((1-alpha)*prob) + bias)               # teleportation formula applied to every page that is not equal to 0  
    
        for x in connectedPages:
            row[x] = prob                              # the pages probabilities stored in the list of row

        for y in range(len(row)):                      
            if row[y] == 0:
                row[y] += bias                         # teleportation formula applied to every page that is equal to 0

        matrix.append(row)                             # appends the row to the matrix

    trans_matrix = np.array(matrix)                    # convert the list of rows to a numpy array so it can be reshaped
    trans_matrix.reshape(7,7)                          # reshaped the array to be 
    
    
    return trans_matrix

In [15]:
trans_matrix = pagerank(edges, 0.14, 100)

dataframe=pd.DataFrame(trans_matrix, columns=['0','1','2', '3', '4', '5', '6']) #Put into a dataframe for an easier visalisation 
dataframe

,0,1,2,3,4,5,6
0,0.020,0.020000,0.880000,0.020000,0.020000,0.02,0.020000
1,0.020,0.306667,0.306667,0.020000,0.020000,0.02,0.306667
2,0.235,0.020000,0.235000,0.235000,0.020000,0.02,0.235000
3,0.020,0.020000,0.020000,0.306667,0.306667,0.02,0.306667
4,0.020,0.020000,0.020000,0.020000,0.020000,0.02,0.880000
5,0.020,0.020000,0.020000,0.020000,0.020000,0.45,0.450000
6,0.020,0.020000,0.020000,0.306667,0.306667,0.02,0.306667


### Function which returns the amount of pages each page links to:

In [16]:
# outgoing_dict = dictionary with pages as keys and their frequencies as values
# edge = the current edge from the list of edges which is being utilised

outgoing_dict = dict.fromkeys(range(number_of_pages),0)

for page in range(number_of_pages):
    for x in range(number_of_edges):
        edge = list_of_edges[x]                 # Gets the specific edge from the list

        if edge[0] == page:                     # Checks the first value as this is where the edge comes from
            outgoing_dict[page] += 1 

outgoing_dict

{0: 1, 1: 3, 2: 4, 3: 3, 4: 1, 5: 2, 6: 3}